Assignment 1 : Spam Filter
Student name : Balasubramanian A.R
Batch : M.Sc Computer science, second year

In [1]:
import pandas as pd
import numpy as np

#Importing tf-idf, Logisitic Regresssion, SVM, Decision Tree and Naive Bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Loading data from the given 'tsv' file
df = pd.read_csv("SMSSpamCollection", delimiter = "\t", names=["label", "sms"])

In [2]:
X = df['sms']
y = df['label']

#Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Logistic Regression involves doing the following computation: We assume that we have a $n$-dimensional vector called the 'weight' vector $w$ and a number $b$. Given an input vector $x$, we first compute the dot product $w \cdot x + b = z$. Once we have $z$, we compute $p(x) = \frac{1}{(1+e^{-z})}$. Notice that this $p$ always lies in $[0,1]$. We then classify $x$ depending on whether $p >= 0.5$ or not.

Let $C(x)$ be the binary classification of an input vector $x$. We need to find $w, b$ s.t. for positive examples, i.e, $x$ s.t. $C(x) = 1$, we need to maximize $\frac{1}{(1+e^{-(w \cdot x + b)})}$ and for negative examples, i.e, $x$ s.t. $C(x) = 0$, we need to maximize $1 - \frac{1}{(1+e^{-(w \cdot x + b)})}$. Therefore, whenever $y = C(x) = 1$, we need to maximize $p(x)$ and whenever $y = C(x) = 0$, we need to maximize $1-p(x)$. Hence we will find $w$ s.t. the following error function is minimized: $$\sum_{i = 1}^n -[y \log (p(x)) + (1-y) \log (1-p(x))]$$ where the summation is over the training examples, and $y$ is the label of each example.

In [3]:
#Logisitic Regression
logreg_pipeline = Pipeline([('vectorizer', TfidfVectorizer()),
                     ('classifier', LogisticRegression())])

In [4]:
print("The cross validation scores are: {}".format(cross_val_score(logreg_pipeline, X=X_train, y=y_train, cv=5)))

logreg_pipeline.fit(X_train, y_train)

/home/arbalan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The cross validation scores are: [0.97132616 0.97129187 0.95454545 0.96407186 0.95449102]


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [5]:
logreg_predicted = logreg_pipeline.predict(X_test)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_test, logreg_predicted)))
print("The number of correctly classified samples is {}\n".format(accuracy_score(y_test, logreg_predicted, normalize=False)))

print("Confusion matrix")
print(pd.DataFrame(confusion_matrix(y_test, logreg_predicted),
             index=['pred ham', 'pred spam'],
            columns=['true ham','true spam']))
print("\n")

print(classification_report(y_test, logreg_predicted))

The fraction of correctly classified samples is 0.9734386216798278
The number of correctly classified samples is 1356

Confusion matrix
           true ham  true spam
pred ham       1207          0
pred spam        37        149


              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1207
        spam       1.00      0.80      0.89       186

   micro avg       0.97      0.97      0.97      1393
   macro avg       0.99      0.90      0.94      1393
weighted avg       0.97      0.97      0.97      1393



Support Vector Machines (SVM) are a method for performing supervised learning. Suppose one class of vectors is lablled by 1 and the other class by -1. We would like to find a "weight" vector $w$ and a real number $b$ s.t.
the following holds for each training example: $y_i(w \cdot x + b) \ge 1$ and $w \cdot w$ is minimized. Once we have this, we can classify a test example by doing the following $C(x) = sgn((w \cdot x) + b)$. The $w$ here acts as a hyperplane which separates the positive and negative examples.

Unfortunately, we might not always be able to find such a hyperplane. In particular if the data is not linearly separable, we might not be able to find one. Hence, we relax the constraints by saying that the following should hold for every training example, $\exists \; \zeta_i \ge 0$ s.t. $y_i(w \cdot x + b) \ge 1 - \zeta_i$ and 
$w \cdot w + \sum_{i=1}^n \zeta_i$ is minimized. The $\zeta_i$'s act as "slack" variables to the original formulation. 

Notice that the $w$ we find is still a hyperplane. If the data is highly non-linear, this might give us a poor classification. Hence, we transform the input vectors to a vector in a "feature space", which is typically a higher dimensional vector compared to the input space. The hope is that in the "feature space", the data becomes linearly separable. Hence, working with the dual of the above formulation and if $\phi$ is a map from the input space to the feature space, then we maximize:
$$\sum_{i=1}^n \alpha_i - \sum_{i=1}^n \sum_{j=1}^n y_i y_j \alpha_i \alpha_j (\phi(x_i) \cdot \phi(x_j))$$

subject to $\sum_{i=1}^n y_i\alpha_i = 0, 0 \le \alpha_i \le 1$. To make this formulation more tractable, we won't explicitly compute the map $\phi$, but will instead use a function $K(x_i,x_j)$ which will return $\phi(x_i) \cdot \phi(x_j)$, i.e. $K(x_i,x_j)$ will represent the inner product in the feature space. This is called as the kernel trick. A commonly used kernel is the polynomial kernel which is : $K(x_i,x_j) = (x_i \cdot x_j)^d$ for some $d$. The case when $d = 1$ is called the linear kernel.

In [6]:
#SVM with linear kernel
SVM_pipeline = Pipeline([('vectorizer', TfidfVectorizer()),
                     ('classifier', SVC(kernel = 'linear'))])

In [7]:
print("The cross validation scores are: {}".format(cross_val_score(SVM_pipeline, X=X_train, y=y_train, cv=5)))

SVM_pipeline.fit(X_train, y_train)

The cross validation scores are: [0.99283154 0.98564593 0.9784689  0.98323353 0.98323353]


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...r', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [8]:
SVM_predicted = SVM_pipeline.predict(X_test)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_test, SVM_predicted)))
print("The number of correctly classified samples is {}\n".format(accuracy_score(y_test, SVM_predicted, normalize=False)))

print("Confusion matrix")
print(pd.DataFrame(confusion_matrix(y_test, SVM_predicted),
             index=['pred ham', 'pred spam'],
            columns=['true ham','true spam']))
print("\n")

print(classification_report(y_test, SVM_predicted))

The fraction of correctly classified samples is 0.9913854989231874
The number of correctly classified samples is 1381

Confusion matrix
           true ham  true spam
pred ham       1207          0
pred spam        12        174


              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1207
        spam       1.00      0.94      0.97       186

   micro avg       0.99      0.99      0.99      1393
   macro avg       1.00      0.97      0.98      1393
weighted avg       0.99      0.99      0.99      1393



Decision trees are one of the commonly used supervised learning techniques. A decision tree consists of a tree where each node is labelled by some attribute of the input. Depending on the attribute value of the input vector at the current node, the tree branches into multiple children and the above procedure is repeated once again. All the leaves are marked with either a $0$ or $1$, which will denote the classification of the given input. 

A decision tree is built from the training data as follows. Initially, we start with a tree of single node. At each node, we have to pick an attribute to branch on. This attribute can be selected based on various measures. One such commonly used measure is the information gain, which is calculated as follows: Let $ent(T)$ be the entropy of the whole dataset, i.e, $ent(T) = -\left (\frac{n_1}{n} \cdot \log(\frac{n_1}{n}) + \frac{n_2}{n} \cdot  \log(\frac{n_2}{n}) \right)$ where $n$ is the size of the training set and $n_1, n_2$ are the number of training examples labelled positive and negative respectively. We then pick an attribute $A_i$ and partition $T$ into $T_1,\cdots,T_k$ s.t. each $T_i$ contains training examples having the same attribute value. Once this is done, we compute $ent_{A_i}(T) = \sum_{j=1}^k \frac{|T_j|}{n} \cdot ent(T_j)$. We now define the gain to be $gain_{A_i} = ent(T) - ent_{A_i}(D)$. Intuitively, the higher the gain of an attribute, the more "purer" each partition $T_1,\cdots,T_k$ becomes. Once gain is computed for each $A_i$, we pick the attribute with the most gain. We now create $k$ children each with dataset $T_1,\cdots,T_k$ and then recurse till we reach a dataset with only one classification, which we will label as a leaf.

In [9]:
#Decision Tree
DTC_pipeline = Pipeline([('vectorizer', TfidfVectorizer()),
                     ('classifier', DTC())])

In [10]:
print("The cross validation scores are: {}".format(cross_val_score(DTC_pipeline, X=X_train, y=y_train, cv=5)))

DTC_pipeline.fit(X_train, y_train)

The cross validation scores are: [0.95340502 0.95933014 0.95334928 0.95928144 0.95688623]


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [11]:
DTC_predicted = DTC_pipeline.predict(X_test)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_test, DTC_predicted)))
print("The number of correctly classified samples is {}\n".format(accuracy_score(y_test, DTC_predicted, normalize=False)))

print("Confusion matrix")
print(pd.DataFrame(confusion_matrix(y_test, DTC_predicted),
             index=['pred ham', 'pred spam'],
            columns=['true ham','true spam']))
print("\n")

print(classification_report(y_test, DTC_predicted))

The fraction of correctly classified samples is 0.9676956209619526
The number of correctly classified samples is 1348

Confusion matrix
           true ham  true spam
pred ham       1192         15
pred spam        30        156


              precision    recall  f1-score   support

         ham       0.98      0.99      0.98      1207
        spam       0.91      0.84      0.87       186

   micro avg       0.97      0.97      0.97      1393
   macro avg       0.94      0.91      0.93      1393
weighted avg       0.97      0.97      0.97      1393



Naive Bayes is a supervised learning algorithm based on the Bayes rule. Suppose the set of features we have are $x_1,\cdots,x_n$ and we want to predict the class labele $y$ of this feature vector. Bayes' rule tells us that

$$P(y|x_1,\cdots,x_n) = \frac{P(y) P(x_1,\cdots,x_n|y)}{P(x_1,\cdots,x_n)}$$

We assume that $P(x_i|y,x_{i+1},x_{i+2},\cdots,x_n) = P(x_i|y)$, i.e, the features are independent of each other. This seems to be a very strong assumption on the dataset, but empricially this works well in practice. Once, we have this assumption, the above expression simplifies to

$$P(y|x_1,\cdots,x_n) = \frac{P(y) \prod_{i=1}^n P(x_i|y)}{P(x_1,\cdots,x_n)}$$

To classify a given feature vector $z_1,\cdots,z_n$ in the case of binary classification, we will first compute $P(y|z_1,\cdots,z_n)$ for both $y = 0,1$ and return the label with the maximum value. Notice that since the denominator of these two values will be the same, we need not compute the denomiator.

Hence, it suffices to show how to estimate the probabilities $P(y)$ and $P(x_i|y)$ from the training data.
We simply take $P(y) = n_y/n$, where $n_y$ is the number of training samples classified as label $y$ and $n$ is the total number of training samples. Similarly, we take $P(x_i|y)$ to be $n_{i,y}/n_y$, where $n_{i,y}$ is the number of training samples with the $i^{th}$ attribute value as $x_i$ and label as $y$. Once we have this computation, we can classify according to the scheme mentioned above.

In [12]:
#Naive Bayes
Bayes_pipeline = Pipeline([('vectorizer', TfidfVectorizer()),
                     ('classifier', MultinomialNB())])

In [13]:
print("The cross validation scores are: {}".format(cross_val_score(Bayes_pipeline, X=X_train, y=y_train, cv=5)))

Bayes_pipeline.fit(X_train, y_train)

The cross validation scores are: [0.95459976 0.95813397 0.95454545 0.9508982  0.94131737]


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...      vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [14]:
Bayes_predicted = Bayes_pipeline.predict(X_test)

print("The fraction of correctly classified samples is {}".format(accuracy_score(y_test, Bayes_predicted)))
print("The number of correctly classified samples is {}\n".format(accuracy_score(y_test, Bayes_predicted, normalize=False)))

print("Confusion matrix")
print(pd.DataFrame(confusion_matrix(y_test, Bayes_predicted),
             index=['pred ham', 'pred spam'],
            columns=['true ham','true spam']))
print("\n")

print(classification_report(y_test, Bayes_predicted))

The fraction of correctly classified samples is 0.9641062455132807
The number of correctly classified samples is 1343

Confusion matrix
           true ham  true spam
pred ham       1207          0
pred spam        50        136


              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1207
        spam       1.00      0.73      0.84       186

   micro avg       0.96      0.96      0.96      1393
   macro avg       0.98      0.87      0.91      1393
weighted avg       0.97      0.96      0.96      1393



Out of the four classifiers, we notice that an SVM with the linear kernel gives the highest accuracy as well as high precision and recall rates. This leads us to conclude that the tf-idf vectorization combined with an SVM of linear kernel is the best classifier for the dataset.